In [14]:
import json
import time
import requests
from bs4 import BeautifulSoup
from collections import deque

def find_shortest_path(start, end):
    path_from_start = {start: [start]}
    path_from_end = {end: [end]}
    Q_from_start = deque([start])
    Q_from_end = deque([end])

    while Q_from_start and Q_from_end:
        page_from_start = Q_from_start.popleft()
        page_from_end = Q_from_end.popleft()

        links_from_start = get_links(page_from_start)
        links_from_end = get_links(page_from_end)

        # check for intersection from start to end
        for link in links_from_start:
            if link in path_from_end:
                return path_from_start[page_from_start] + path_from_end[link][::-1]

            if (link not in path_from_start) and (link != page_from_start):
                path_from_start[link] = path_from_start[page_from_start] + [link]
                Q_from_start.append(link)

        # check for intersection from end to start
        for link in links_from_end:
            if link in path_from_start:
                return path_from_start[link] + path_from_end[page_from_end][::-1]

            if (link not in path_from_end) and (link != page_from_end):
                path_from_end[link] = path_from_end[page_from_end] + [link]
                Q_from_end.append(link)

    return None

def get_links(page):
    r = requests.get(page)
    soup = BeautifulSoup(r.content, 'html.parser')
    base_url = page[:page.find('/wiki/')]
    links = list({base_url + a['href'] for a in soup.select('p a[href]') if a['href'].startswith('/wiki/')})
    return links

def check_pages(start, end):
    languages = []
    for page in [start, end]:
        try:
            ind = page.find('.wikipedia.org/wiki/')
            languages.append(page[(ind-2):ind])
            requests.get(page)
        except:
            print(f'{page} does not appear to be a valid Wikipedia page.')
            return False

    if len(set(languages)) > 1:
        print('Pages are in different languages.')
        return False

    if len(get_links(start)) == 0:
        print('Start page is a dead-end page with no Wikipedia links.')
        return False

    end_soup = BeautifulSoup(requests.get(end).content, 'html.parser')
    if end_soup.find('table', {'class': 'metadata plainlinks ambox ambox-style ambox-Orphan'}):
        print('End page is an orphan page with no Wikipedia pages linking to it.')
        return False
    return True

def redirected(end):
    end_soup = BeautifulSoup(requests.get(end).content, 'html.parser')
    title = end_soup.find('h1').text
    title = title.replace(' ', '_', len(title))
    base_url = end[:end.find('/wiki/') + len('/wiki/')]
    return set([end, base_url + title])

def result(start, end, path):
    if path:
        result = path
    else:
        result = "No path! :( "
    d = {"start": start, "end": end, "path": result}
    return json.dumps(d, indent=4)

def main(input_json):
    data = json.loads(input_json)
    start = data["start"]
    end = data["end"]
    if check_pages(start, end):
        path = find_shortest_path(start, end)
        json_result = result(start, end, path)
        return json_result


In [15]:
input_json = '{"start": "https://en.wikipedia.org/wiki/2023_South_Korea_floods", "end": "https://en.wikipedia.org/wiki/Artificial_intelligence"}'
starttime = time.time()

print(main(input_json))

endtime = time.time()
totaltime = endtime - starttime
print('Time: {}m {:.3f}s'.format(int(totaltime)/60, totaltime%60))

{
    "start": "https://en.wikipedia.org/wiki/2023_South_Korea_floods",
    "end": "https://en.wikipedia.org/wiki/Artificial_intelligence",
    "path": [
        "https://en.wikipedia.org/wiki/2023_South_Korea_floods",
        "https://en.wikipedia.org/wiki/Joseon",
        "https://en.wikipedia.org/wiki/Wikipedia:Citation_needed",
        "https://en.wikipedia.org/wiki/Rodney_Brooks",
        "https://en.wikipedia.org/wiki/Artificial_intelligence"
    ]
}
Time: 0.03333333333333333m 2.354s


In [16]:
import json
import time
import requests
from bs4 import BeautifulSoup
from collections import deque

def find_shortest_path(start, end):
    path = {}
    path[start] = [start]
    Q = deque([start])
    while len(Q) != 0:
        page = Q.popleft()
        links = get_links(page)
        for link in links:
            if link in end:
                return path[page] + [link]
            if (link not in path) and (link != page):
                path[link] = path[page] + [link]
                Q.append(link)
    return None

def get_links(page):
    r = requests.get(page)
    soup = BeautifulSoup(r.content, 'html.parser')
    base_url = page[:page.find('/wiki/')]
    links = list({base_url + a['href'] for a in soup.select('p a[href]') if a['href'].startswith('/wiki/')})
    return links

def check_pages(start, end):
    languages = []
    for page in [start, end]:
        try:
            ind = page.find('.wikipedia.org/wiki/')
            languages.append(page[(ind-2):ind])
            requests.get(page)
        except:
            print(f'{page} does not appear to be a valid Wikipedia page.')
            return False

    if len(set(languages)) > 1:
        print('Pages are in different languages.')
        return False

    if len(get_links(start)) == 0:
        print('Start page is a dead-end page with no Wikipedia links.')
        return False

    end_soup = BeautifulSoup(requests.get(end).content, 'html.parser')
    if end_soup.find('table', {'class': 'metadata plainlinks ambox ambox-style ambox-Orphan'}):
        print('End page is an orphan page with no Wikipedia pages linking to it.')
        return False
    return True

def redirected(end):
    end_soup = BeautifulSoup(requests.get(end).content, 'html.parser')
    title = end_soup.find('h1').text
    title = title.replace(' ', '_', len(title))
    base_url = end[:end.find('/wiki/') + len('/wiki/')]
    return set([end, base_url + title])

def result(start, end, path):
    if path:
        result = path
    else:
        result = "No path! :( "
    d = {"start": start, "end": end, "path": result}
    return json.dumps(d, indent=4)

def main(input_json):
    data = json.loads(input_json)
    start = data["start"]
    end = data["end"]
    if check_pages(start, end):
        path = find_shortest_path(start, redirected(end))
        json_result = result(start, end, path)
        return json_result


In [ ]:
input_json = '{"start": "https://en.wikipedia.org/wiki/2023_South_Korea_floods", "end": "https://en.wikipedia.org/wiki/Artificial_intelligence"}'
starttime = time.time()

print(main(input_json))

endtime = time.time()
totaltime = endtime - starttime
print('Time: {}m {:.3f}s'.format(int(totaltime)/60, totaltime%60))
